In [25]:
#!pip -q install langchain openai tiktoken

In [22]:
import os
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [45]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

## Wikipedia

In [46]:
#!pip -q install wikipedia

In [47]:
from langchain.utilities import WikipediaAPIWrapper

In [48]:
wikipedia = WikipediaAPIWrapper()

In [49]:
wikipedia.run('Langchain')

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: Sentence embedding\nSummary: In natural language processing, a sentence embedding refers to a numeric representation of a sentence in the form of a vectors of real numbers which encodes meaningful semantic information.State of the art embeddings are based on the learned hidden layer representation of dedicated sentence transformer models. BERT pioneered an approach involving the use of a dedicated [CLS] token preprended to the beginning of each sentence inputted into the model; the final hidden state vector of this token encodes information about the sentence and can be fine-tuned for use in sentence classification tasks. In practice however,

## REPL

In [50]:
from langchain.utilities import PythonREPL

In [51]:
python_repl = PythonREPL()

In [52]:
python_repl.run("print(17*2)")

'34\n'

## DuckDuckGo

In [58]:
!pip -q install duckduckgo-search

In [78]:
query = "who whon the 1990 world cup"

In [79]:
from itertools import islice
from duckduckgo_search import DDGS

class DDG(DDGS):
    def run (query:str):
        '''
        "snippet": result["body"],
        "title": result["title"],
        "link": result["href"],
        '''
        results = DDGS().text(query)
        generator = [result['body'] for result in results]

        # Get the first three results from the generator
        first_three_results = list(islice(generator, 4))
        return first_three_results

In [80]:
from langchain import OpenAI

llm = OpenAI(temperature=0)

In [81]:
from langchain.agents import Tool

tools = [
    Tool(
        name = "python repl",
        func=python_repl.run,
        description="useful for when you need to use python to answer a question. You should input python code"
    ),
    Tool(
        name='wikipedia',
        func= wikipedia.run,
        description="Useful for when you need to look up a topic, country or person on wikipedia"
    ),
    Tool(
        name='DuckDuckGo Search',
        func= DDG.run,
        description="Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
)]

In [82]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
)

In [83]:
zero_shot_agent.run("When was Michael Saylor born?")



> Entering new  chain...
 I need to find out when Michael Saylor was born.
Action: Wikipedia
Action Input: Michael Saylor
Observation: Wikipedia is not a valid tool, try another one.
Thought: I need to find out when Michael Saylor was born.
Action: DuckDuckGo Search
Action Input: Michael Saylor birthdate
Observation: ['Life Saylor was born in Lincoln, Nebraska on February 4, 1965, and spent his early years on various Air Force bases around the world, as his father was an Air Force chief master sergeant. When Saylor was 11, the family settled in Fairborn, Ohio, near the Wright-Patterson Air Force Base. [3] [4]', "Birthdate: February 4, 1965 Age: 58 years old (as of 2023) Ethnicity: Unknown Nationality: American Sexuality: N/A Michael Saylor was born on the 4th of February, 1965. He is best known for being a Entrepreneur. Saylor and fellow business executive and educational entrepreneur Sal Khan both graduated from MIT. Michael Saylor's age is 58.", '#1818 Billionaires (2022) Dropped o

'Michael Saylor was born on February 4, 1965.'